# Telemetry Collection Agent (TCA) loads via COA
# ResUsageSPMA

In [64]:
%disconnect TranscendDev

Success: 'TranscendDev' disconnected

In [65]:
%connect Transcend

Password: ············


Success: 'Transcend' connection established and activated for user 'sh186014'


In [66]:
SET TIME ZONE 'America Pacific';

Success: 1 rows affected

In [67]:
-- super DBCInfo
Select InfoKey, InfoData from dbc.dbcinfo
union all 
Select 'SystemType', SystemType FROM  TABLE (SYSLIB.MonitorSystemPhysicalConfig()) as CFG
union all 
Select 'SystemName', SystemName FROM  TABLE (SYSLIB.MonitorSystemPhysicalConfig()) as CFG
union all 
select 'Current Time', nowish from (select Current_Timestamp(0)(varchar(25)) as nowish) as n 
order by 1

InfoKey,InfoData
Current Time,2020-11-25 13:45:40-08:00
LANGUAGE SUPPORT MODE,Standard
RELEASE,16.20.53.06
SystemName,TRANSCEND02
SystemType,7011
VERSION,16.20.53.06


<br><br><br>
# Build objects required to "catch" COA data
```
LOGICAL STEP       DATABASE       TABLE (per version)          DOES WHAT?
GTT Stage table    app_tca_tmp    stg_tca_resusageSPMA_xxyy    Catches raw data  
Target Table       app_tca_tbl    coa_tca_resusageSPMA_xxyy    Long-term storage
Stored Proc        app_tca_tbl    coa_tca_resusageSPMA_xxyy    Merges stg to tgt
View of Target     app_tca_vw      sp_tca_resusageSPMA_xxyy    Access View on Target
```

<br><br><br>
## SQL for generating upload file

In [ ]:
-- SQL for generating .csv file (PDCR):
select '{siteid}' as Site_ID, a.*
from pdcrinfo.resusageSPMA as a
where TheDate between DATE-1 and DATE

<br>

## APP_TCA_TMP -- STAGE Table -- _1620

In [105]:
database adlste_coa_stg;

Success: 1 rows affected

In [106]:
drop table stg_tca_resusageSPMA_1620

Success: 278 rows affected

In [87]:
/* GLOBAL TEMP TABLE for stage data coming from COA
   note: all users of ADLSTE_COA datalab will need 
         SELECT, INSERT, DELETE, and UPDATE to this object */
CREATE SET GLOBAL TEMPORARY TABLE stg_tca_resusageSPMA_1620  as
(
 SELECT ''(varchar(128)) as Site_ID, a.*  
 FROM raw_wla_vw.resusageSPMA_hst_1620 as a
) 
with no data
primary index (Site_ID, TheDate, NodeID)
on commit preserve rows;

Success: 0 rows affected

### Drop audit columns to return to inital source table state

In [88]:
ALTER TABLE stg_tca_resusageSPMA_1620 DROP tca_system_id;

Success: 2 rows affected

In [89]:
ALTER TABLE stg_tca_resusageSPMA_1620 DROP di_batch_nbr;

Success: 2 rows affected

In [90]:
ALTER TABLE stg_tca_resusageSPMA_1620 DROP di_proc_name;

Success: 2 rows affected

In [91]:
ALTER TABLE stg_tca_resusageSPMA_1620 DROP di_create_ts;

Success: 2 rows affected

In [92]:
ALTER TABLE stg_tca_resusageSPMA_1620 DROP di_update_ts;

Success: 2 rows affected

## Unit test:

In [93]:
insert into stg_tca_resusageSPMA_1620
SELECT top 10 'SiteID', r.* FROM pdcrinfo.ResusageSPMA_hst as r

Success: 10 rows affected

In [ ]:
-- remove audit colums from GTT table (should match SEL*)
ALTER TABLE APP_TCA_TMP.stg_tca_resusageSPMA_1620 DROP tca_system_id;

In [ ]:
ALTER TABLE APP_TCA_TMP.stg_tca_resusageSPMA_1620 DROP di_batch_nbr;

In [ ]:
ALTER TABLE APP_TCA_TMP.stg_tca_resusageSPMA_1620 DROP di_proc_name;

In [ ]:
ALTER TABLE APP_TCA_TMP.stg_tca_resusageSPMA_1620 DROP di_create_ts;

In [ ]:
ALTER TABLE APP_TCA_TMP.stg_tca_resusageSPMA_1620 DROP di_update_ts;

In [24]:
-- TEST:
INSERT INTO APP_TCA_TMP.stg_tca_resusageSPMA_1620
Select 'TestSite', a.* From pdcrinfo.resusageSPMA as a
where TheDate = DATE-1

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3523:The user does not have SELECT access to PDCRINFO.ResUsageSpma.

<br>

## APP_TCA_TBL -- TARGET Table

In [97]:
database adlste_coa;

Success: 1 rows affected

In [99]:
drop Table coa_tca_resusageSPMA_1620

Success: 282 rows affected

In [100]:
-- TARGET TABLE:
CREATE SET GLOBAL TEMPORARY TABLE coa_tca_resusageSPMA_1620  as
(
 SELECT ''(varchar(128)) as Site_ID, a.*  
 FROM raw_wla_vw.resusageSPMA_hst_1620 as a
) 
with no data
primary index (Site_ID, TheDate, NodeID)
on commit preserve rows;

Success: 0 rows affected

## APP_TCA_VW.coa_tca_dbcinfo

In [101]:
database adlste_coa;

Success: 1 rows affected

In [103]:
-- ABSTRACTION LAYER VIEW  (** OPTIONAL **) --
replace view coav_tca_resusageSPMA_1620 as locking row for access
Select * from coa_tca_resusageSPMA_1620;

Success: 0 rows affected

## APP_TCA_TBL.sp_tca_dbcinfo

In [ ]:
drop procedure APP_TCA_TBL.sp_tca_dbcinfo

In [ ]:
REPLACE PROCEDURE APP_TCA_TBL.sp_tca_dbcinfo () 
SQL SECURITY CREATOR
BEGIN     /* this is a simplified example, excluding any audit logging */ 

    DECLARE proc_name CHAR(30);  
    DECLARE batch_nbr INTEGER ;  
    
    /* someone will need to fill these values from ETL */
    SET proc_name = 'Unknown';
    SET batch_nbr = 0        ;   
   
    
    MERGE INTO APP_TCA_TBL.coat_tca_dbcinfo  tgt
         USING APP_TCA_TMP.stg_tca_dbcinfo   stg
            ON (tgt.Site_ID = stg.Site_ID and
                tgt.InfoKey = stg.InfoKey)
    WHEN MATCHED THEN 
        UPDATE SET 
              InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_upd_ts      = Current_Timestamp(6)
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr
    WHEN NOT MATCHED THEN 
        INSERT VALUES 
            ( Site_ID        = stg.Site_ID
            , InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_created_ts  = Current_Timestamp(6)
            , di_upd_ts      = NULL
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr ) 
    ;

    delete from APP_TCA_TMP.stg_tca_dbcinfo ;
        
END;

<br><br>
# Unit Test:
run this a few times over, to emulate repeat runs

In [ ]:
insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

In [ ]:
call APP_TCA_TBL.sp_tca_dbcinfo();

In [ ]:
-- should see upd_ts move, while created_ts remains static
Select * from APP_TCA_VW.coa_tca_dbcinfo
order by 1,2;

<br><br>
## Again:

In [ ]:
insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

In [ ]:
call APP_TCA_TBL.sp_tca_dbcinfo();

In [ ]:
-- should see upd_ts move, while created_ts remains static
Select * from APP_TCA_VW.coa_tca_dbcinfo
order by 1,2;